Project Euler 77
---
Prime summation
It is possible to write ten as the sum of primes in exactly five different ways:

7+3, 2+2+2+2+2, 3+3+2+2, 5+3+2, 5+5

What is the first value which can be written as the sum of primes in over five thousand different ways?
---

First, if a number $n = p_1p_2$, then it can be written as a sum of $p_2$ $p_1$s and $p_1$ $p_2$s. So any non-prime has at least 2 of these sums.

There is probably a good way to build up possible sums from previous numbers.

Alternatively, we can approach from the other direction. Generate all the numbers that are made by adding primes in different ways.

In [4]:
import numpy as np
import sympy as sp
import pandas as pd

In [3]:
sp.primefactors(100)

[2, 5]

In [4]:
#list a bunch of prime numbers
primes200 = [ sp.prime(nth) for nth in range(1,200)]
#create a bunch of numbers by adding primes.
#num_to_add[i] is how many ith primes to add
num_to_add = np.zeros_like(primes200)

In [82]:
# def increase_by(times_to_add, current_focused_index = 0, max_no = 100):
#     new_numbos = times_to_add.copy()
#     searching_for_first = True
#     current_index = 0
#     num_numbos = len(times_to_add)    
#     while searching_for_first:
#         if times_to_add[current_index] == max_no:
#             current_index += 1
#         elif times_to_add[current_index] < max_no:
#             new_numbos[current_index] += 1
#             return new_numbos
#         if current_index == len(num_numbos):
#             searching_for_first = True

# #returns a times_to_add increased by one, 
# def increase_by(times_to_add, current_focused_index = 0, max_no = 100):
#     # print(f'increase {times_to_add} at {current_focused_index}...)+')
#     new_numbos = times_to_add.copy()
#     if new_numbos[current_focused_index] == max_no:
#         new_numbos[current_focused_index] = 0
#         current_focused_index += 1
#         if current_focused_index == len(times_to_add):
#             return np.zeros_like(times_to_add), current_focused_index
#     new_numbos[current_focused_index] = new_numbos[current_focused_index] + 1
#     return new_numbos, current_focused_index
        
# def all_times_to_add(size, max_no = 10 ):
#     times_to_add = np.zeros(size)
#     all_times = [times_to_add.copy()]
#     current_focused_index = 0
#     while current_focused_index < size:
#         times_to_add, current_focused_index = increase_by(times_to_add, current_focused_index = current_focused_index, max_no = max_no)
#         all_times.append(times_to_add)
#     return all_times

def all_ways(size, upper_bound):
    if size == 1:
        return np.arange(max_no):

Lets start with 5000+ "prime markers" and distribute them amongst the first... 200 primes? Fewer? More? See what numbers we get. Include zero for down to 5000.

How many different ways can we fill N prime boxes with S things?
Each thing can go into one of N, so N^S if they are distinguishable. Divide by every rearrangement of the things once they are placed.
$$
\frac{N^S}{N!}
$$

In [10]:
N = 10
S = 5001
(N**S)/np.math.perm(N)

OverflowError: integer division result too large for a float

Sep 17

Can think of the vector of all primes dotted with a vector of counts:
$$
P\cdot S = N\\
\left[2,3,5,7,\dots p_n\right]\cdot\left[s_1, s_2, \dots s_n\right] = N
$$

The vector S is the solution --- there should be at least 5000 of these that work in the positive "quadrant".

Mapping all the ways every number can be made of other primes should be a good first step. Then we just need to multiply ways given those primes to get many ways to write them.

In [59]:
primes200[:10]

10

In [58]:
max_sum_multiples = [ 5000, 4000, 3500, 2500, 2500, 250, 250, 25, 20, 10]

9

In [68]:
def numberToBase(n, b):
    if n == 0:
        return [0]
    digits = []
    while n:
        digits.append(int(n % b))
        n //= b
    return digits#[::-1]

In [83]:
def extend_list(the_list, full_length):
    list_length = len(the_list)
    new_list = the_list + [0]*(full_length - list_length)
    return new_list

def make_record(multiples, primes = primes200):
    extended_multiples = extend_list(multiples, len(primes))
    the_sum = np.matmul(extended_multiples, primes)
    record = [the_sum, multiples, sum(multiples)]
    return record

In [170]:
data = []
for number in range(10000000):
    multiples = numberToBase(number, 200)
    data.append(make_record(multiples, primes = primes200[:7]))
df = pd.DataFrame(data, columns = ['sum value','prime sums','number primes used'])

In [166]:
pd.options.display.max_rows = 60

grouped_df = df.groupby('sum value').count()
grouped_df = grouped_df[['prime sums']]
grouped_df[(grouped_df['prime sums'] > 300)]

,prime sums
sum value,
130,304
131,308
132,313
133,317
134,322
...,...
856,322
857,317
858,313


In [167]:
grouped_df.max()

prime sums    2000
dtype: int64

In [169]:
grouped_df[grouped_df['prime sums'] >= 1800]

,prime sums
sum value,
386,1802
387,1806
388,1809
389,1813
390,1816
...,...
600,1816
601,1813
602,1809


sept 18
---


Limit the number of times we can add a prime so it itself cannot exceed a certain number.
With $N$, each prime $p_i$ cannot be added more than $N/p$ times. So the sums can greatly exceed $N$, as we can add other primes $N/p_n$ times as well.

In [342]:
max_num = 35
max_prime_times = np.array(list(map(lambda p: max_num//p, primes200)))
np.product(max_prime_times[max_prime_times != 0] + 1)

5971968

There are 6 Million different ways to add the first few primes so none on their own exceeds 35.

In [309]:
max_prime_times = max_prime_times[max_prime_times != 0]

In [310]:
max_prime_times

array([17, 11,  7,  5,  3,  2,  2,  1,  1,  1,  1])

In [313]:
primes11=primes200[:len(max_prime_times)]

In [314]:
max_prime_times

array([17, 11,  7,  5,  3,  2,  2,  1,  1,  1,  1])

In [315]:
primes11

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31]

In [316]:
def increment_prime_times(prime_times, prime_index = 0, maxes = max_prime_times):
    if prime_times[prime_index] == maxes[prime_index]:
        if prime_index == len(prime_times) - 1:
            return np.zeros_like(prime_times)
        else:
            prime_times[prime_index] = 0
            return increment_prime_times(prime_times, prime_index = prime_index + 1)
    else:
        prime_times[prime_index] = prime_times[prime_index] + 1
        return prime_times
        

In [322]:
max_prime_times

array([17, 11,  7,  5,  3,  2,  2,  1,  1,  1,  1])

In [317]:
increment_prime_times(np.zeros_like(max_prime_times))

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [318]:
all_prime_times[0][1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [325]:
all_prime_times = []
prime_times = np.zeros_like(max_prime_times)
all_prime_times.append(
    (0, prime_times.copy())
)
for number in range(1,np.product(max_prime_times + 1)): #need to +1 to the product because 0 is an option as well.
    increment_prime_times(prime_times)
    all_prime_times.append(
        (number, prime_times.copy())
    )
    
data = [
    (all_prime_time[1], np.matmul(all_prime_time[1], primes15))
    for all_prime_time in all_prime_times
]

ValueError: Length of values (5971968) does not match length of index (78540)

In [326]:

df = pd.DataFrame(data, columns = ['prime_multiples', 'sum'], index = range(np.product(max_prime_times+1)))

In [327]:
df.head()

,prime_multiples,sum
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",2
2,"[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",4
3,"[3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",6
4,"[4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",8


In [328]:
df.tail(60)

,prime_multiples,sum
5971908,"[12, 8, 7, 5, 3, 2, 2, 1, 1, 1, 1]",313
5971909,"[13, 8, 7, 5, 3, 2, 2, 1, 1, 1, 1]",315
5971910,"[14, 8, 7, 5, 3, 2, 2, 1, 1, 1, 1]",317
5971911,"[15, 8, 7, 5, 3, 2, 2, 1, 1, 1, 1]",319
5971912,"[16, 8, 7, 5, 3, 2, 2, 1, 1, 1, 1]",321
5971913,"[17, 8, 7, 5, 3, 2, 2, 1, 1, 1, 1]",323
5971914,"[0, 9, 7, 5, 3, 2, 2, 1, 1, 1, 1]",292
5971915,"[1, 9, 7, 5, 3, 2, 2, 1, 1, 1, 1]",294
5971916,"[2, 9, 7, 5, 3, 2, 2, 1, 1, 1, 1]",296
5971917,"[3, 9, 7, 5, 3, 2, 2, 1, 1, 1, 1]",298


In [329]:
df['sum'].value_counts()

166    57960
167    57944
165    57944
164    57895
168    57895
       ...  
0          1
2          1
3          1
4          1
332        1
Name: sum, Length: 331, dtype: int64

In [338]:
dfvcs = df['sum'].value_counts()
dfvcsind = dfvcs.index
dfvcsmask = (dfvcs > 5000) & (dfvcs < 5500)
dfvcs[dfvcsmask]

77     5223
255    5223
Name: sum, dtype: int64

In [340]:
df[df['sum'] == 77]

,prime_multiples,sum
647,"[17, 11, 2, 0, 0, 0, 0, 0, 0, 0, 0]",77
844,"[16, 10, 3, 0, 0, 0, 0, 0, 0, 0, 0]",77
1041,"[15, 9, 4, 0, 0, 0, 0, 0, 0, 0, 0]",77
1074,"[12, 11, 4, 0, 0, 0, 0, 0, 0, 0, 0]",77
1205,"[17, 6, 5, 0, 0, 0, 0, 0, 0, 0, 0]",77
...,...,...
4482450,"[0, 1, 0, 2, 0, 0, 0, 0, 0, 1, 1]",77
4489347,"[3, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1]",77
4489380,"[0, 2, 0, 0, 1, 0, 0, 0, 0, 1, 1]",77
4520450,"[2, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1]",77


In [343]:
#only those adding up to the max number or less include every possibility.
df[df['sum'] <= 35]

,prime_multiples,sum
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",2
2,"[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",4
3,"[3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",6
4,"[4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",8
...,...,...
1493208,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0]",34
2985984,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",31
2985985,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",33
2985986,"[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",35


In [344]:
df35 = df[df['sum'] <= 35]

In [345]:
df35['sum'].value_counts()

35    175
34    157
33    140
32    124
31    111
30     98
29     87
28     77
27     67
26     60
25     52
24     46
23     40
22     35
21     30
20     26
19     23
18     19
17     17
16     14
15     12
14     10
13      9
12      7
11      6
10      5
9       4
8       3
7       3
5       2
6       2
4       1
3       1
2       1
0       1
Name: sum, dtype: int64

In [351]:
#there are 175 ways to add prime numbers to 35, etc.
df35[df35['sum'] == 10]

,prime_multiples,sum
5,"[5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",10
38,"[2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0]",10
235,"[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]",10
432,"[0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0]",10
1746,"[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0]",10


Number of ways to add primes to N:
N = 2 + (N-2)
Number of ways = 2 + number of ways (N-2)

In [361]:
print(df35[df35['sum'].apply(lambda x: x in primes200)]['sum'].value_counts().to_markdown())

|    |   sum |
|---:|------:|
| 31 |   111 |
| 29 |    87 |
| 23 |    40 |
| 19 |    23 |
| 17 |    17 |
| 13 |     9 |
| 11 |     6 |
|  7 |     3 |
|  5 |     2 |
|  2 |     1 |
|  3 |     1 |


|  prime number  |   number of ways to add primes |
|---:|------:|
| 31 |   111 |
| 29 |    87 |
| 23 |    40 |
| 19 |    23 |
| 17 |    17 |
| 13 |     9 |
| 11 |     6 |
|  7 |     3 |
|  5 |     2 |
|  2 |     1 |
|  3 |     1 |

First, any number can be written as a unique sum of 2s and 3s.

$$
N = \left\{
\begin{array}{ll}
2(N // 2) & N \text{ even}\\
2(N // 2 -1 ) + 3 & N \text{ odd}
\end{array}
\right\}
$$

All prime numbers greater than 2 are odd, so all primes can be expressed as the second form.
So, to get a prime in the sum, pick out a number of 2s, and a 3, then check if prime.

This means that any prime number must have at least as many ways of writing it as primes below do (?) Maybe not. depends on whether 2s and 3s or whatever show up in the expansions.

$31 = 14 \times 2 +3 = 28 + 3$

28 isn't prime, but can be expanded into prime sums.

number = $\left[ \text{ primes } \right] \cdot \left[ \text{ multiples } \right]$

5000 different ways of distributing integers amongst $\text{multiples}$ to get the same number.



Say N is the first number that can be written like this, at least 5000 different ways $W_i$.

$W$ stands for Way.

$$
\left[ \text{ primes } \right] \cdot W_1 = \left[ \text{ primes } \right] \cdot W_2 = \dots = \left[ \text{ primes } \right] \cdot W_{5000} = N
$$
And all of $W_i$ are distinct.

Expand primes to diagonalized?

$$
\left[ \begin{array}{cccc}p & 0& & 0\\0& r&\dots&\\\vdots&0&\ddots&\vdots\\ 0&&& p_i\end{array} \right] \left[ 1\ 1\ \dots 1\right] \ W_1 = \left[ \text{ primes } \right] \cdot W_2 = \dots = \left[ \text{ primes } \right] \cdot W_{5000} = N
$$

In [2]:
import numpy as np

In [11]:
lfivehun_primes = list(sp.primerange(1,501))

W_5000 vectors of some $p$ primes, all different...

Points in a grid in $p$ dimensions....

Don't matrixify the primes! Matrixify the Ws.

$$
PW =\Biggl[ \text{p r i m e s }\dots P_p \Biggr] \Biggl[W_1\ W_2\ \dots W_{5000} \Biggr]  = N \bar 1
$$

with $\bar 1$ the vector of ones. $P$ is shaped $1\times p$, $W$ is shaped $p\times 5000$, and the right is a 1 by 5000 vector of ones times the Number N.

This is kind of like finding a null space... but it's the ones space.